In [3]:
"""
https://towardsdatascience.com/noise-its-not-always-annoying-1bd5f0f240f
Std dev = 4
Loss: 2.304307460784912 Accuracy: 0.09375
Final test accuracy: 0.11349999904632568
"""

"""
A standard deviation of 4 is too high and adds too much noise into the data. 
The average standard deviation observed  observed in internet is between 0 and 1


Std Dev = 1
Loss: 0.7794443368911743 Accuracy: 0.75
Final test accuracy: 0.9099000096321106
"""
"""
Std Dev = 0.5
Loss: 0.4842979907989502 Accuracy: 0.84375
Final test accuracy: 0.9297999739646912
"""
"""
Std Dev = 0.25
Loss: 0.33627036213874817 Accuracy: 0.87890625
Final test accuracy: 0.9330000281333923
"""

'\nLoss: 2.304307460784912 Accuracy: 0.09375\nFinal test accuracy: 0.11349999904632568\n'

In [6]:
import tensorflow as tf
import os
from datasets import MNISTDataset
import time
logdir = os.path.join("logs", "linear" + str(time.time()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))


# get the data
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.mnist.load_data()
mnist = MNISTDataset(train_imgs.reshape((-1, 784)), train_lbls,
                     test_imgs.reshape((-1, 784)), test_lbls,
                     batch_size=256, seed=int(time()))


# define the model first, from input to output

# let's use fewer layers...
n_units = 100
n_layers = 2

# just set up a "chain" of hidden layers
layers = []
for layer in range(n_layers):
    layers.append(tf.keras.layers.Dense(
        n_units, activation=tf.nn.relu,
        kernel_initializer=tf.initializers.RandomUniform(minval=-0.01,
                                                         maxval=0.01),
        bias_initializer=tf.initializers.constant(0.001)))

# finally add the output layer
layers.append(tf.keras.layers.Dense(
    10, kernel_initializer=tf.initializers.RandomUniform(minval=-0.01,
                                                         maxval=0.01)))

lr = 0.1
for step in range(2000):
    img_batch, lbl_batch = mnist.next_batch()
    # I hear adding random noise to inputs helps with generalization!
    img_batch = img_batch + tf.random.normal(tf.shape(img_batch), stddev=0.25)
    with tf.GradientTape() as tape:
        # here we just run all the layers in sequence via a for-loop
        out = img_batch
        for layer in layers:
            out = layer(out)
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=out, labels=lbl_batch))

    weights = [var for l in layers for var in l.trainable_variables]
    grads = tape.gradient(xent, weights)
    for grad, var in zip(grads, weights):
        var.assign_sub(lr*grad)
    i=0        
    with train_writer.as_default():
        tf.summary.scalar("loss", xent, step=step)
        tf.summary.histogram("logits", out, step=step)
        for W in weights:
            tf.summary.histogram("weights"+str(i), W, step=step)
            i+=1
    tf.summary.scalar("Loss", xent, step=step)
    if not step % 100:
        preds = tf.argmax(out, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch), tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))
        with train_writer.as_default():
            tf.summary.scalar("Trainaccuracy", acc, step=step)
            tf.summary.image("input", tf.reshape(img_batch, [-1, 28, 28, 1]), step=step)

out = mnist.test_data
for layer in layers:
    out = layer(out)
test_preds = tf.argmax(out, axis=1, output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, mnist.test_labels), tf.float32))
with test_writer.as_default():
    tf.summary.scalar("Testaccuracy", acc, step=step)
print("Final test accuracy: {}".format(acc))


Loss: 2.302563428878784 Accuracy: 0.12890625
Loss: 2.3007941246032715 Accuracy: 0.140625
Loss: 2.293738842010498 Accuracy: 0.15234375
Starting new epoch...
Loss: 2.2997817993164062 Accuracy: 0.1015625
Loss: 2.299534559249878 Accuracy: 0.1015625
Starting new epoch...
Loss: 2.283576011657715 Accuracy: 0.13671875
Loss: 1.8336962461471558 Accuracy: 0.34765625
Loss: 1.125978946685791 Accuracy: 0.59375
Starting new epoch...
Loss: 0.8605972528457642 Accuracy: 0.72265625
Loss: 0.690257728099823 Accuracy: 0.78125
Starting new epoch...
Loss: 0.5305319428443909 Accuracy: 0.85546875
Loss: 0.6198354363441467 Accuracy: 0.81640625
Starting new epoch...
Loss: 0.4431857466697693 Accuracy: 0.875
Loss: 0.46280303597450256 Accuracy: 0.8671875
Loss: 0.3465435802936554 Accuracy: 0.91015625
Starting new epoch...
Loss: 0.3944286108016968 Accuracy: 0.86328125
Loss: 0.410513699054718 Accuracy: 0.8828125
Starting new epoch...
Loss: 0.43435025215148926 Accuracy: 0.88671875
Loss: 0.4067035913467407 Accuracy: 0.882

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs